### We gonna use 3 differentes models to test our data


##### They are 
##### - multi-qa-distilbert-cos-v1 (SentenceTransformer)
##### - sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 (SentenceTransformer)
##### - cross-encoder/ms-marco-MiniLM-L-6-v2 (CrossEncoder)

In [3]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import CrossEncoder
import torch
model = SentenceTransformer("multi-qa-distilbert-cos-v1")


/home/vanderson/miniconda3/envs/bert/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
2024-09-27 14:42:18.765507: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-27 14:42:18.783584: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-27 14:42:18.789145: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-27 14:42:18.802702: I tensorflow/core/platform/cpu_f

In [4]:
# Carregar o modelo sentence-transformers multilíngue
model_LM = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


In [7]:

model_cross = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2", default_activation_function=torch.nn.Sigmoid(),max_length=64)



In [53]:
##Using the first model (multi_qa) as the model is not trained in multilingual
## and we want to use to portuguese phrases, i will have to translate to use this model
passage_embeddings=model.encode([
    "cancelar consulta",
   " cancelar uma consulta",
    "Quero cancelar a consulta",
   " Como faz para cancelar a consulta",
   " Preciso cancelar a consulta",
    "Gostaria de desmarcar a consulta",
   " Como faço para cancelar a consulta que marquei"
])
query_embedding=model.encode("quero cancelar minha consulta")
similarity=model.similarity(query_embedding,passage_embeddings)
print(similarity)

tensor([[0.8022, 0.7465, 0.8904, 0.8691, 0.8514, 0.5310, 0.7641]])


In [54]:
##As we can see, the scores values to english phrases are much higher
passage_embeddings=model.encode([
    "cancel appointment", 
    "cancel an appointment", 
    "I want to cancel the appointment", 
    "How do I cancel the appointment", 
    "I need to cancel the appointment", 
    "I would like to cancel the appointment", 
    "How do I cancel the appointment I made"
])
query_embedding=model.encode("I want to cancel my appointment")
similarity=model.similarity(query_embedding,passage_embeddings)
print(similarity)

tensor([[0.9259, 0.9335, 0.9778, 0.9416, 0.9662, 0.9486, 0.9275]])


In [5]:
### USING THE SECOND MODEL, MULTILINGUAL
### Testing with portuguese phrases
query_embedding=model_LM.encode("Quero cancelar uma consulta com o psicologo")
# Exemplo de uso com frases em português
passage_embeddings=model_LM.encode([
    "Quero cassar uma consulta com o psicologo"
])
similarity=model_LM.similarity(query_embedding,passage_embeddings)

print(similarity)

tensor([[0.7247]])


In [85]:
#### For english phrases, we do not see much difference is scores
query_embedding=model_LM.encode("I want to book an appointment")
passage_embeddings=model_LM.encode([
"book an appointment with a psychologist",
"book an appointment",
"how do I schedule an appointment",
"how do I schedule an appointment with you",
"I want to schedule an appointment with you",
"how do I schedule an appointment with you",
"How can I book an appointment",
"Is it possible to schedule an appointment"
])
similarity=model_LM.similarity(query_embedding,passage_embeddings)

print(similarity)

tensor([[0.5216, 0.8320, 0.7826, 0.7398, 0.7807, 0.7398, 0.9449, 0.7662]])


In [84]:
### Lets compare the scores with the first model
query_embedding=model.encode("I want to book an appointment")
passage_embeddings=model.encode([
"book an appointment with a psychologist",
"book an appointment",
"how do I schedule an appointment",
"how do I schedule an appointment with you",
"I want to schedule an appointment with you",
"how do I schedule an appointment with you",
"How can I book an appointment",
"Is it possible to schedule an appointment"
])
similarity=model.similarity(query_embedding,passage_embeddings)

print(similarity)

tensor([[0.5282, 0.9095, 0.7804, 0.7978, 0.8564, 0.7978, 0.9507, 0.7263]])


In [ ]:
#stsb roberta is a good model too
from sentence_transformers import CrossEncoder
import torch
model_cross = CrossEncoder('cross-encoder/stsb-roberta-base', 
                           default_activation_function=torch.nn.Sigmoid(), 
                           max_length=64)


##### the first model obtained higher scores for English sentences

In [11]:
### Lets test the cross encoder model
scores = model_cross.predict([
    ("como agendo uma consulta", "marcar uma consulta")

])
print(scores)

[0.81391525]


#### This cross encoder model does not handle synonymous words very well, so it will not be very useful for data augmentation, since the intention is to expand the database with synonyms and similar words, maintaining the meaning of the sentence.

### But, how cross encoder detect a high level of score with similar sentences, we are gonna use to detect duplicates or too similar phrases in our dataset, to avoid too much similar data